In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # 会提示授权，点击链接并复制验证码

Mounted at /content/drive


In [ ]:
   !pip install lightgbm openpyxl --quiet

In [ ]:
!pip install lightgbm openpyxl
from google.colab import files
uploaded = files.upload()  # 上传您的文件

Saving 副本论文数据.xlsx to 副本论文数据 (1).xlsx


In [ ]:
from pandas import ExcelFile
excel = ExcelFile("副本论文数据.xlsx")
for sheet in excel.sheet_names:
    df = pd.read_excel("副本论文数据.xlsx", sheet_name=sheet)
    print(f"\nSheet: {sheet}")
    print("列名:", df.columns.tolist())
    print("前2行数据:\n", df.head(2))


Sheet: 旅游物价
列名: ['時期 (排序)', '旅遊及服務', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']
前2行数据:
   時期 (排序)                       旅遊及服務  Unnamed: 2 Unnamed: 3  Unnamed: 4  \
0     NaN  旅遊物價指數 (10/2009 - 09/2010)         NaN        NaN         NaN   
1     NaN                         總指數         NaN        NaN         NaN   

  Unnamed: 5  Unnamed: 6 Unnamed: 7  Unnamed: 8 Unnamed: 9  Unnamed: 10  \
0        NaN         NaN        NaN         NaN        NaN          NaN   
1        NaN         NaN        NaN         NaN        NaN          NaN   

  Unnamed: 11  
0         NaN  
1     總指數年變動率  

Sheet: 旅客非博彩消费
列名: ['Unnamed: 0', '旅遊及服務', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10']
前2行数据:
   Unnamed: 0    旅遊及服務 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0        NaN       旅遊        NaN        NaN       

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def load_and_clean_data(file_path):
    """
    加载并清洗Excel数据，处理年度和季度数据的合并
    参数：
        file_path: Excel文件路径
    返回：
        合并清洗后的DataFrame
    """
    # 定义工作表映射关系
    sheets = {
        'concerts': '演唱会',         # 年度数据
        'non_gaming': '旅客非博彩消费',  # 季度数据
        'tourism_price': '旅游物价指数', # 季度数据
        'hotel_occ': '酒店平均入住率',   # 季度数据
        'cross_border': '跨界汽车量',    # 季度数据
        'gaming': '博彩收入'           # 季度数据
    }

    dfs = {}  # 存储各表数据

    try:
        print(">>> 正在加载演唱会数据（年度）...")
        # 1. 处理演唱会数据（年度）
        concerts = pd.read_excel(file_path, sheet_name=sheets['concerts'])
        # 标准化列名
        concerts.columns = ['年份', '演唱会数量', 'Unnamed_2', '观众人数']
        # 提取年份并转换为日期（每年1月1日）
        concerts['日期'] = pd.to_datetime(
            concerts['年份'].str.replace('年', '') + '-01-01',
            errors='coerce'
        )
        # 只保留需要的列
        dfs['concerts'] = concerts[['日期', '演唱会数量', '观众人数']]

        # 2. 加载并处理季度数据
        quarterly_sheets = {
            'non_gaming': ('旅客非博彩消费', ['时期', '非博彩收入_百万澳门元', '旅客人均消费_澳门元']),
            'tourism_price': ('旅游物价指数', ['时期', '旅游物价指数']),
            'hotel_occ': ('酒店平均入住率', ['时期', '酒店平均入住率_百分比', '酒店住宿消费']),
            'cross_border': ('跨界汽车量', ['时期', '跨境汽车流量_车次']),
            'gaming': ('博彩收入', ['时期', '博彩收入_百万澳门元'])
        }

        for key, (sheet_name, cols) in quarterly_sheets.items():
            print(f">>> 正在加载 {sheet_name} 数据（季度）...")
            df = pd.read_excel(file_path, sheet_name=sheet_name)

            # 检查列数是否匹配
            if len(df.columns) != len(cols):
                print(f"警告: {sheet_name} 表列数不匹配，原始列名: {list(df.columns)}")
                # 尝试自动处理多余的列
                df = df.iloc[:, :len(cols)]

            # 重命名列
            df.columns = cols

            # 转换季度日期格式（如"2023年第1季" -> 2023-01-01）
            df['日期'] = df['时期'].apply(
                lambda x: convert_quarter_date(x) if pd.notnull(x) else pd.NaT
            )

            # 删除原始日期列
            df = df.drop('时期', axis=1)

            # 存储处理后的数据
            dfs[key] = df.dropna(subset=['日期'])  # 删除日期无效的行

        print("\n>>> 正在合并所有数据...")
        # 3. 合并数据（特殊处理年度到季度的转换）
        # 创建基础时间索引（所有季度日期）
        min_year = dfs['concerts']['日期'].min().year
        max_year = dfs['concerts']['日期'].max().year
        all_dates = pd.date_range(
            start=f'{min_year}-01-01',
            end=f'{max_year}-12-31',
            freq='QS'  # 季度开始日期
        )

        # 创建基础DataFrame
        merged_df = pd.DataFrame({'日期': all_dates})

        # 添加年份和季度列
        merged_df['年份'] = merged_df['日期'].dt.year
        merged_df['季度'] = merged_df['日期'].dt.quarter.map({1:'Q1', 2:'Q2', 3:'Q3', 4:'Q4'})

        # 合并演唱会数据（年度->季度）
        concerts_yearly = dfs['concerts'].copy()
        concerts_yearly['年份'] = concerts_yearly['日期'].dt.year
        concerts_yearly = concerts_yearly.drop('日期', axis=1)

        merged_df = pd.merge(
            merged_df,
            concerts_yearly,
            on='年份',
            how='left'
        )

        # 合并其他季度数据
        for key in quarterly_sheets.keys():
            merged_df = pd.merge(
                merged_df,
                dfs[key],
                on='日期',
                how='left'
            )

        print("\n>>> 正在清洗数据...")
        # 4. 数据清洗和类型转换
        # 定义数值列
        numeric_cols = [
            '演唱会数量', '观众人数', '非博彩收入_百万澳门元',
            '旅客人均消费_澳门元', '旅游物价指数', '酒店平均入住率_百分比',
            '酒店住宿消费', '跨境汽车流量_车次', '博彩收入_百万澳门元'
        ]

        # 转换数值类型
        for col in numeric_cols:
            if col in merged_df.columns:
                merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

        # 排序并重置索引
        merged_df = merged_df.sort_values('日期').reset_index(drop=True)

        # 重新排序列
        base_cols = ['日期', '年份', '季度']
        other_cols = [c for c in merged_df.columns if c not in base_cols]
        merged_df = merged_df[base_cols + other_cols]

        return merged_df

    except Exception as e:
        print(f"\n!!! 数据处理出错: {str(e)}")
        return None

def convert_quarter_date(date_str):
    """
    将中文季度日期转换为标准日期
    示例：
        "2023年第1季" -> 2023-01-01
        "2022年第3季" -> 2022-07-01
    """
    try:
        if pd.isnull(date_str):
            return pd.NaT

        # 处理季度格式
        if '年第' in date_str and '季' in date_str:
            year = int(date_str.split('年')[0])
            quarter = int(date_str.split('第')[1].replace('季', ''))

            # 计算季度起始月份
            month_map = {1:1, 2:4, 3:7, 4:10}
            return datetime(year, month_map[quarter], 1)

        # 处理其他格式（如有需要可以扩展）
        else:
            return pd.to_datetime(date_str, errors='coerce')
    except:
        return pd.NaT

# 使用示例
if __name__ == "__main__":
    file_path = '副本论文数据.xlsx'
    print(">>> 开始处理数据...")
    cleaned_data = load_and_clean_data(file_path)

    if cleaned_data is not None:
        print("\n>>> 数据处理成功！")
        print("数据形状:", cleaned_data.shape)
        print("\n处理后的数据样例：")
        print(cleaned_data.head(100))  # 显示前8行查看年度到季度的转换

        # 保存结果
        output_path = '整合后的旅游数据_年度季度统一.xlsx'
        cleaned_data.to_excel(output_path, index=False)
        print(f"\n>>> 数据已保存为 '{output_path}'")

        # 添加格式说明
        print("\n数据结构说明：")
        print("- 每行代表一个季度（Q1-Q4）")
        print("- 年度数据（如演唱会）自动扩展到四个季度")
        print("- 所有数值列已转换为适当的数据类型")
    else:
        print(">>> 数据处理失败，请检查原始数据格式。")

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib

# 改style要在改font之前
# plt.style.use('seaborn')

matplotlib.font_manager.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
matplotlib.rc('font', family='Taipei Sans TC Beta')

--2025-08-13 06:28:54--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 64.233.181.101, 64.233.181.138, 64.233.181.113, ...
Connecting to drive.google.com (drive.google.com)|64.233.181.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2025-08-13 06:28:54--  https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 64.233.181.132, 2607:f8b0:4001:c09::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|64.233.181.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20659344 (20M) [application/octet-stream]
Saving to: ‘TaipeiSansTCBeta-Regular.ttf’

TaipeiSansTCBeta-Re 100%[===================>]  19.70M  --.-KB/s    in 0.09s   

2025-08-13 06:28:

In [ ]:
%matplotlib inline

In [ ]:
# -*- coding: utf-8 -*-
"""澳门演唱会经济效益预测模型（最终问题修复版）"""
from google.colab import drive, files
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
import traceback
from sklearn.model_selection import TimeSeriesSplit

# --------------------------
# 0. 数据加载与探索
# --------------------------
def load_data(filepath):
    """加载数据并进行初步探索"""
    try:
        print(">>> 正在加载数据...")
        df = pd.read_excel(filepath, sheet_name='Sheet1')

        # 检查必需字段
        required_cols = ['日期', '演唱会数量', '观众人数', '非博彩收入_百万澳门元']
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            print("当前数据列名:", df.columns.tolist())
            raise ValueError(f"缺失必要字段: {missing_cols}")

        print("\n数据加载成功！数据概况:")
        print(f"数据集形状: {df.shape}")
        print(f"数据时间段: {df['日期'].min()} 至 {df['日期'].max()}")

        # 显示所有列的数据类型
        print("\n数据类型:")
        print(df.dtypes)

        # 显示数值列的基本统计
        numeric_cols = df.select_dtypes(include=np.number).columns
        if not numeric_cols.empty:
            print("\n数值列统计摘要:")
            print(df[numeric_cols].describe().transpose())

        # 显示缺失值情况
        print("\n缺失值统计:")
        print(df.isnull().sum())

        return df

    except Exception as e:
        raise RuntimeError(f"数据加载失败: {str(e)}")

# --------------------------
# 1. 数据预处理 (修改版)
# --------------------------
def preprocess_data(df):
    """数据清洗与转换 - 修复数据分布问题"""
    print("\n>>> 数据预处理中...")

    # 保存原始数据副本
    original_df = df.copy()

    # 转换日期类型并提取时间特征
    df['日期'] = pd.to_datetime(df['日期'], errors='coerce')
    df['年份'] = df['日期'].dt.year
    df['月份'] = df['日期'].dt.month
    df['季度'] = df['日期'].dt.quarter
    df['星期'] = df['日期'].dt.dayofweek

    # 定义特征类型 (提前定义)
    numerical_features = ['演唱会数量', '观众人数', '非博彩收入_百万澳门元',
                         '酒店平均入住率_百分比', '零售业销售额_百万澳门元',
                         '旅客人均消费_澳门元', '入境旅客人数']

    categorical_features = ['规模等级', '演唱会类型', '主要客源地']

    # 修复规模等级分布问题
    if '观众人数' in df.columns:
        print("\n修复规模等级分布问题...")
        # 计算实际分位数
        q1 = df['观众人数'].quantile(0.33)
        q2 = df['观众人数'].quantile(0.67)

        # 保存为全局变量以便后续使用
        global SIZE_BINS, SIZE_LABELS
        SIZE_BINS = [0, q1, q2, float('inf')]
        SIZE_LABELS = ['小型', '中型', '大型']

        # 重新创建规模等级
        df['规模等级'] = pd.cut(df['观众人数'],
                              bins=SIZE_BINS,
                              labels=SIZE_LABELS)



        # 确保有足够样本
        size_counts = df['规模等级'].value_counts()
        print("规模等级分布:", size_counts)

        # 如果某种规模缺失，添加合成数据
        for size in SIZE_LABELS:
            if size not in size_counts or size_counts[size] < 5:
                print(f"警告: {size}演唱会样本不足，添加合成数据")
    # 创建合成数据逻辑...

    # 处理缺失值
    print("\n缺失值处理前:")
    print(df.isnull().sum())

    # 数值列 - 中位数填充
    for col in numerical_features:
        if col in df.columns and df[col].isnull().any():
            median_val = df[col].median()
            print(f"填充数值列 '{col}' 的缺失值: 中位数 = {median_val}")
            df[col] = df[col].fillna(median_val)

    # 分类列 - 众数填充
    for col in categorical_features:
        if col in df.columns and df[col].isnull().any():
            mode_val = df[col].mode()[0]
            print(f"填充分类列 '{col}' 的缺失值: 众数 = {mode_val}")
            df[col] = df[col].fillna(mode_val)

    print("\n缺失值处理后:")
    print(df.isnull().sum())

    # 检查是否还有缺失值
    if df.isnull().sum().sum() > 0:
        print("\n警告: 数据中仍存在缺失值!")
        print(df.isnull().sum())
        # 使用前向填充作为最后手段
        df = df.fillna(method='ffill').fillna(method='bfill')
        print("\n前向/后向填充后:")
        print(df.isnull().sum())

    # 创建特征相关性矩阵
    plt.figure(figsize=(12, 10))
    corr_matrix = df.select_dtypes(include=np.number).corr()
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
    plt.title('特征相关性矩阵', fontsize=14)
    plt.tight_layout()
    plt.savefig('特征相关性.png', dpi=300)
    plt.show()

    return df, original_df, numerical_features, categorical_features

# --------------------------
# 2. 特征工程 (修复版)
# --------------------------
def feature_engineering(df, numerical_features, categorical_features):
    """构建高级特征 - 修复分类特征处理问题"""
    print("\n>>> 特征工程处理中...")

    # 基础特征：使用所有可用特征
    exclude_cols = ['日期', '非博彩收入_百万澳门元']
    all_features = [col for col in df.columns if col not in exclude_cols]

    # 创建关键交互特征 - 增强演唱会效应
    if all(col in df.columns for col in ['演唱会数量', '酒店平均入住率_百分比']):
        df['演唱会_酒店交互'] = df['演唱会数量'] * df['酒店平均入住率_百分比']
        all_features.append('演唱会_酒店交互')
        numerical_features.append('演唱会_酒店交互')  # 新增数值特征

    if all(col in df.columns for col in ['观众人数', '旅客人均消费_澳门元']):
        df['观众消费潜力'] = df['观众人数'] * df['旅客人均消费_澳门元'] / 1000  # 转换为千澳门元
        all_features.append('观众消费潜力')
        numerical_features.append('观众消费潜力')  # 新增数值特征

    # 创建滞后特征 - 捕捉持续效应
    if '非博彩收入_百万澳门元' in df.columns:
        df['上期非博彩收入'] = df['非博彩收入_百万澳门元'].shift(1)
        all_features.append('上期非博彩收入')
        numerical_features.append('上期非博彩收入')  # 新增数值特征

    # 目标变量
    target = '非博彩收入_百万澳门元'

    # ===== 关键修复：处理分类特征编码 =====
    # 1. 单独提取分类特征
    categorical_cols = list(set(all_features) & set(categorical_features))
    if categorical_cols:
        print(f"正在处理分类特征: {categorical_cols}")

        # 创建标签编码器
        label_encoders = {}

        # 将分类特征映射为数值
        for col in categorical_cols:
            print(f"编码列: {col}")

            # 获取唯一值并创建映射
            unique_vals = df[col].unique()
            # 特殊处理：如果只有两个值，可以直接转换为0/1
            if len(unique_vals) == 2:
                mapping = {val: idx for idx, val in enumerate(unique_vals)}
            else:
                # 为每个类别赋值（排序后）
                sorted_vals = sorted(unique_vals)
                mapping = {val: idx for idx, val in enumerate(sorted_vals)}

            # 应用映射
            df[col] = df[col].map(mapping)
            label_encoders[col] = mapping

            print(f"  映射关系: {mapping}")

            # 编码后这些特征应视为数值特征
            if col in categorical_features:
                categorical_features.remove(col)
            numerical_features.append(col)

   # 2. 准备特征矩阵
    X = df[all_features]
    y = df[target]

    # 3. 处理缺失值
    # 数值特征 - 中值填充
    numerical_cols_in_X = list(set(all_features) & set(numerical_features))
    for col in numerical_cols_in_X:
        if X[col].isnull().any():
            median_val = X[col].median()
            X[col] = X[col].fillna(median_val)
            print(f"填充数值特征 '{col}' 的中位值: {median_val}")

    # 4. 确保没有剩余NaN值
    if X.isnull().any().any():
        print("警告: 特征数据仍有NaN值，使用中值填充")
        X = X.fillna(X.median())

    # 添加特征相关性检查
    print("\n关键特征相关性检查:")
    if target in df.columns:
        # 创建临时DF用于相关性检查
        temp_df = X.copy()
        temp_df[target] = y

        # 计算相关性矩阵
        corr_matrix = temp_df.corr()
        print(corr_matrix[target].sort_values(ascending=False))

    return X, y, all_features, categorical_features, label_encoders

# --------------------------
# 3. 模型训练 (修复版)
# --------------------------
def train_model(X, y, categorical_features):
    """模型训练与评估 - 修复分类特征处理"""
    print("\n>>> 开始模型训练...")

    # 最终检查NaN
    if X.isnull().any().any() or y.isnull().any():
        print("警告: 输入数据仍包含NaN值!")
        print("特征数据NaN:", X.isnull().sum().sum())
        print("目标变量NaN:", y.isnull().sum())

        # 填充剩余的NaN
        X = X.fillna(X.median())
        y = y.fillna(y.median())
        print("NaN值已填充")

    # 数据集分割
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

   # ===== 关键修复：确保分类特征一致性 =====
    # 不再使用分类特征索引，因为所有特征都已数值化
    cat_features = []

    print(f"分类特征索引: {cat_features}")

    # 创建数据集 - 不再指定分类特征
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


    # 模型参数
    params = {
        'objective': 'regression',
        'metric': 'mape',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_data_in_leaf': 10,
        'verbosity': -1,
        'seed': 42,
        'force_col_wise': True
    }

    # 训练模型
    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[test_data],
        callbacks=[
            lgb.early_stopping(100, verbose=True),
            lgb.log_evaluation(100)
        ]
    )

    # 评估
    y_pred = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    mae = mean_absolute_error(y_test, y_pred)

    print(f"\n模型性能评估:")
    print(f"平均绝对百分比误差(MAPE): {mape:.2f}%")
    print(f"平均绝对误差(MAE): {mae:,.2f} 百万澳门元")

    # 残差分析
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    plt.scatter(y_pred, residuals, alpha=0.6)
    plt.axhline(y=0, color='r', linestyle='-')
    plt.xlabel('预测值', fontsize=12)
    plt.ylabel('残差', fontsize=12)
    plt.title('残差分析', fontsize=14)
    plt.grid(True)
    plt.savefig('残差分析.png', dpi=300)
    plt.show()

    return model, X_test, y_test, X

# --------------------------
# 4. 结果分析
# --------------------------
def analyze_results(model, X, features, df, original_df):
    """深入分析模型结果"""
    print("\n>>> 生成分析结果...")

    # 特征重要性
    plt.figure(figsize=(12, 8))
    lgb.plot_importance(model, max_num_features=15, importance_type='gain', figsize=(12, 8))
    plt.title('特征重要性排序', fontsize=16)
    plt.tight_layout()
    plt.savefig('特征重要性.png', dpi=300)
    plt.show()

    # 预测效果
    y_pred = model.predict(X)
    plt.figure(figsize=(12, 8))
    plt.plot(original_df['日期'], original_df['非博彩收入_百万澳门元'], 'o-', label='实际值', alpha=0.7)
    plt.plot(original_df['日期'], y_pred, 'o-', label='预测值', alpha=0.7)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('非博彩收入（百万澳门元）', fontsize=12)
    plt.title('时间序列预测效果', fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('时间序列预测.png', dpi=300)
    plt.show()

     # SHAP值分析 - 使用解码后的特征名称
    try:
        import shap
        print("\n>>> 生成SHAP解释...")

        # 创建SHAP解释器
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)

        # 特征重要性摘要图
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X, feature_names=features, plot_type="bar", show=False)
        plt.title('SHAP特征重要性', fontsize=16)
        plt.tight_layout()
        plt.savefig('SHAP特征重要性.png', dpi=300)
        plt.show()

        # 单个预测解释
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X, feature_names=features, show=False)
        plt.title('特征影响方向', fontsize=16)
        plt.tight_layout()
        plt.savefig('SHAP特征影响.png', dpi=300)
        plt.show()


    # ===== 修正：计算演唱会实际贡献 =====
    # 创建无演唱会的基准场景
    X_base = X.copy()
    # 将演唱会相关特征设为0
    for col in ['演唱会数量', '观众人数', '演唱会_酒店交互', '观众消费潜力']:
        if col in X_base.columns:
            X_base[col] = 0

    # 预测基准收入（无演唱会）
    base_pred = model.predict(X_base).mean()

    # 预测当前收入
    current_pred = model.predict(X).mean()

    # 计算演唱会带来的额外收入
    concert_effect = current_pred - base_pred

    # 计算平均每场演唱会的贡献
    total_concerts = df['演唱会数量'].mean()
    if total_concerts > 0:
        per_concert_effect = concert_effect / total_concerts
    else:
        per_concert_effect = 0

    print(f"\n演唱会经济效益分析:")
    print(f"基准非博彩收入（无演唱会）: {base_pred:,.2f} 百万澳门元")
    print(f"当前非博彩收入: {current_pred:,.2f} 百万澳门元")
    print(f"演唱会带来的额外收入: {concert_effect:,.2f} 百万澳门元")
    print(f"平均总演唱会数量: {total_concerts:,.0f} 场")
    print(f"平均每场演唱会带动非博彩收入: {per_concert_effect:,.2f} 百万澳门元")

    # 可视化
    plt.figure(figsize=(10, 6))
    labels = ['基准收入（无演唱会）', '当前收入', '演唱会贡献']
    values = [base_pred, current_pred, concert_effect]

    plt.bar(labels, values, color=['gray', 'blue', 'green'])
    plt.title('演唱会经济效益分析', fontsize=14)
    plt.ylabel('非博彩收入（百万澳门元）', fontsize=12)
    plt.xticks(rotation=15)

    # 添加数值标签
    for i, v in enumerate(values):
        plt.text(i, v + 100, f"{v:,.0f}", ha='center')

    plt.tight_layout()
    plt.savefig('演唱会经济效益分析.png', dpi=300)
    plt.show()

    return per_concert_effect
# --------------------------
# 5. 政策模拟 (最终修复版)
# --------------------------
def policy_simulation(model, X, df,label_encoders):
    """修复政策模拟问题 - 确保数据一致性"""
    print("\n=== 政策模拟分析 ===")

   # 获取全局的分箱边界
    global SIZE_BINS, SIZE_LABELS
    print(f"使用分箱边界: {SIZE_BINS}")

    # 获取基础预测值
    base_pred = model.predict(X).mean()
    print(f"基准非博彩收入: {base_pred:,.2f} 万澳门元")

    # 定义更合理的政策场景
    scenarios = [
        ('现状', {}),
        ('增加跨境车量', {'跨境汽车流量_车次': '增加20%', }),
        ('旺季举办大型演唱会', {
           '旺季季度': [1, 4],  # Q1和Q4为旺季
           '旺季演唱会比例': 0.6,     # 60%的演唱会在旺季举办
           '旺季观众增加': 0.3       # 旺季观众增加30%
        }),
        ('提升演唱会配套服务', {
            '旅客人均消费_澳门元': '增加15%',
            '酒店平均入住率_百分比': '增加10%'
        }),
      ('综合策略', {
        '增加跨境车量': '增加15%',
        '旺季季度': [1, 4],
        '旺季演唱会比例': 0.7,
        '旺季观众增加': 0.4,
        '旅客人均消费_澳门元': '增加10%'
        })
    ]
    # 存储政策效果
    policy_names = []
    policy_effects = []
    details = []

    for name, params in scenarios:
        df_scenario = df.copy()
        changes_log = []

    # ===== 特殊处理：旺季举办大型演唱会策略 =====
        if name in ['旺季举办大型演唱会', '综合策略']:
            peak_quarters = params.get('旺季季度', [1, 4])  # Q1和Q4为旺季
            peak_ratio = params.get('旺季演唱会比例', 0.6)
            peak_attendance_boost = params.get('旺季观众增加', 0.3)

             # 1. 计算旺季和非旺季演唱会分配
            total_concerts = df_scenario['演唱会数量'].sum()
            peak_concerts = total_concerts * peak_ratio
            off_peak_concerts = total_concerts * (1 - peak_ratio)

            # 2. 按季度重新分配演唱会
            for idx, row in df_scenario.iterrows():
                quarter = row['季度']  # 使用季度而不是月份
                if quarter in peak_quarters:
                    # 旺季：分配更多演唱会
                    df_scenario.at[idx, '演唱会数量'] = peak_concerts / len(peak_quarters)
                    # 增加观众人数
                    df_scenario.at[idx, '观众人数'] = row['观众人数'] * (1 + peak_attendance_boost)
                else:
                    # 淡季：减少演唱会
                    df_scenario.at[idx, '演唱会数量'] = off_peak_concerts / (4 - len(peak_quarters))

            changes_log.append(f"将{peak_ratio*100:.0f}%演唱会集中在季度{peak_quarters}")
            changes_log.append(f"旺季观众人数增加{peak_attendance_boost*100:.0f}%")

        # 通用参数调整
        for col, val in params.items():
            # 跳过已处理的旺季参数
            if col in ['旺季季度', '旺季演唱会比例', '旺季观众增加']:
                continue

            if col in df_scenario.columns:

                # 处理百分比变化
                if isinstance(val, str) and ('增加' in val or '提升' in val):
                    try:
                        percent = float(val.replace('增加', '').replace('提升', '').replace('%', '').strip())
                        original_mean = df_scenario[col].mean()
                        df_scenario[col] = df_scenario[col] * (1 + percent/100)
                        changes_log.append(f"{col}增加{percent}% (原均值: {original_mean:.2f} → 新均值: {df_scenario[col].mean():.2f})")
                    except ValueError:
                        print(f"无法解析百分比值: {val}")


        # ===== 关键修复：重新计算衍生特征 =====
        # 重新计算规模等级
        if '观众人数' in df_scenario.columns:
            df_scenario['规模等级'] = pd.cut(
                df_scenario['观众人数'],
                bins=SIZE_BINS,  # 使用全局分箱边界
                labels=SIZE_LABELS
            )

        # 重新计算交互特征
        if all(col in df_scenario.columns for col in ['演唱会数量', '酒店平均入住率_百分比']):
            df_scenario['演唱会_酒店交互'] = df_scenario['演唱会数量'] * df_scenario['酒店平均入住率_百分比']

        if all(col in df_scenario.columns for col in ['观众人数', '旅客人均消费_澳门元']):
            df_scenario['观众消费潜力'] = df_scenario['观众人数'] * df_scenario['旅客人均消费_澳门元'] / 1000

        # 分类特征编码 (复用之前的编码器)
        for col, encoder in label_encoders.items():
            if col in df_scenario.columns:
                df_scenario[col] = df_scenario[col].map(encoder)

        # 准备预测数据 - 确保列顺序和类型一致
        X_scenario = df_scenario[X.columns].copy()

        # 确保数据类型与训练数据一致
        for col in X.columns:
            if col in X_scenario.columns:
                X_scenario[col] = X_scenario[col].astype(X[col].dtype)

        # 处理可能的NaN
        if X_scenario.isnull().any().any():
            X_scenario = X_scenario.fillna(X_scenario.median(numeric_only=True))

        # 执行预测
        try:
            scenario_pred = model.predict(X_scenario).mean()
        except Exception as e:
            print(f"预测失败: {str(e)}")
            raise

        effect = scenario_pred - base_pred

        policy_names.append(name)
        policy_effects.append(effect)
        details.append({
            '政策': name,
            '预测收入': scenario_pred,
            '增收': effect,
            '提升比例': effect / base_pred,
            '调整细节': changes_log
        })

        print(f"\n[政策] {name}")
        for log in changes_log:
            print(f"  → {log}")
        print(f"预计收入: {scenario_pred:,.2f} | 增收: {effect:,.2f} | 提升: {effect/base_pred:.2%}")


    # 找出最佳政策
    best_idx = np.argmax(policy_effects)

    # 绘制政策效果图
    plt.figure(figsize=(14, 6))
    colors = ['gray', 'blue', 'green', 'orange', 'purple']

    # 收入对比
    plt.subplot(1, 2, 1)
    bars = plt.bar(policy_names, [d['预测收入'] for d in details], color=colors)
    plt.title('不同政策下的预测收入', fontsize=14)
    plt.ylabel('非博彩收入 (百万澳门元)',  fontsize=14)
    plt.xticks(rotation=15)

    # 添加注释
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height/2,
                 f"{details[i]['提升比例']:.1%}",
                 ha='center', va='center', color='white', fontweight='bold')

    # 增收效果
    plt.subplot(1, 2, 2)
    effect_bars = plt.bar(policy_names[1:], policy_effects[1:], color=colors[1:])
    plt.title('政策增收效果',fontsize=14)
    plt.ylabel('增收金额 (百万澳门元)',fontsize=14)
    plt.xticks(rotation=15)

    for i, bar in enumerate(effect_bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height/2,
                 f"{height:,.0f}",
                 ha='center', va='center', color='white', fontweight='bold')

    plt.tight_layout()
    plt.savefig('政策效果分析.png', dpi=300)
    plt.show()

    return policy_names[best_idx], policy_effects[best_idx], details

# --------------------------
# 6. 间接经济效益评估
# --------------------------
def evaluate_indirect_impact(model, X, df):
    """评估演唱会的间接经济效益"""
    print("\n>>> 评估间接经济效益...")

    # 创建基准场景（无演唱会）
    df_no_concert = df.copy()
    df_no_concert['演唱会数量'] = 0
    df_no_concert['观众人数'] = 0

    # 重置相关特征
    for col in ['演唱会_酒店交互', '观众消费潜力']:
        if col in df.columns:
            df_no_concert[col] = 0

    # 预测无演唱会场景收入
    X_no_concert = df_no_concert[X.columns]
    if X_no_concert.isnull().any().any():
        X_no_concert = X_no_concert.fillna(X_no_concert.median(numeric_only=True))

    no_concert_pred = model.predict(X_no_concert).mean()

    # 计算演唱会总经济效益
    current_pred = model.predict(X).mean()
    total_impact = current_pred - no_concert_pred

    # 按演唱会数量计算平均影响
    avg_concerts = df['演唱会数量'].mean()
    per_concert_impact = total_impact / avg_concerts if avg_concerts > 0 else 0

    print(f"\n演唱会总经济效益: {total_impact:,.2f} 百万澳门元")
    print(f"每场演唱会平均经济效益: {per_concert_impact:,.2f} 百万澳门元")

    # 可视化
    plt.figure(figsize=(10, 6))
    labels = ['当前收入', '无演唱会场景']
    values = [current_pred, no_concert_pred]

    plt.bar(labels, values, color=['green', 'red'])
    plt.title('演唱会经济效益评估')
    plt.ylabel('非博彩收入 (百万澳门元)')

    # 添加差值标注
    plt.annotate(f'+{total_impact:,.0f} 百万',
                xy=(0.5, (current_pred + no_concert_pred)/2),
                xytext=(0.5, (current_pred + no_concert_pred)/2 + 1000),
                arrowprops=dict(arrowstyle='->'),
                ha='center')

    plt.tight_layout()
    plt.savefig('间接经济效益.png', dpi=300)
    plt.show()

    return total_impact, per_concert_impact

# --------------------------
# 主程序 (最终修复版)
# --------------------------
if __name__ == "__main__":
    # 定义全局分箱边界变量
    global SIZE_BINS, SIZE_LABELS
    SIZE_BINS = []  # 初始化为空列表
    SIZE_LABELS = []  # 初始化为空列表

    print("="*60)
    print("澳门演唱会经济效益预测模型 - 完整修复版")
    print("="*60)

    try:
        # 文件上传
        print("\n>>> 请上传数据文件（支持.xlsx）")
        uploaded = files.upload()
        data_file = list(uploaded.keys())[0] if uploaded else None

        if not data_file:
            raise ValueError("请先上传数据文件")

        print(f">>> 使用数据文件: {data_file}")

        # 数据加载
        raw_df = load_data(data_file)

        # 数据预处理
        df_processed, original_df, numerical_features, categorical_features = preprocess_data(raw_df)

        # 保存处理后的数据
        df_processed.to_excel("处理后的数据.xlsx", index=False)
        print("\n>>> 处理后的数据已保存为 '处理后的数据.xlsx'")
        print(f"数值特征: {len(numerical_features)} 个")
        print(f"分类特征: {len(categorical_features)} 个")

        # 特征工程 - 现在返回5个值
        X, y, all_features, categorical_features, label_encoders = feature_engineering(
            df_processed, numerical_features, categorical_features
        )

        # 模型训练 - 不再传递分类特征
        model, X_test, y_test, X_full = train_model(X, y, [])

        # 结果分析 (现在只传5个参数)
        avg_effect = analyze_results(model, X_full, all_features, df_processed, original_df)

        # 政策模拟 - 传递 label_encoders
        best_policy, best_effect, policy_details = policy_simulation(model, X_full, df_processed, label_encoders)  # 添加 label_encoders
        # 间接经济效益评估
        total_impact, per_concert_impact = evaluate_indirect_impact(model, X_full, df_processed)

        # 最终报告
        print("\n" + "="*60)
        print("经济影响分析最终报告")
        print("="*60)
        print(f"经过全面分析，演唱会活动对澳门非博彩收入的带动作用显著")
        print(f"平均每场演唱会可带动非博彩收入: {per_concert_impact:,.2f} 百万澳门元")
        print(f"演唱会活动总体经济效益: {total_impact:,.2f} 百万澳门元")
        print(f"\n最具经济效益的政策方案: {best_policy}")
        print(f"预计可额外增收: {best_effect:,.2f} 百万澳门元")
        print("\n建议澳门政府优先考虑:")
        print("1. 在旅游旺季(Q1和Q4)集中举办大型演唱会")
        print("2. 优化演唱会与酒店、餐饮、零售的联动机制")
        print("3. 加强跨境交通配套，提升旅客体验")
        print("\n>>> 分析完成! 所有图表已保存 <<<")

    except Exception as e:
        print(f"\n!!! 运行出错: {str(e)}")
        traceback.print_exc()

        print("\n问题解决建议:")
        print("1. 检查数据文件格式是否正确")
        print("2. 确保包含必需字段: 日期, 演唱会数量, 观众人数, 非博彩收入_百万澳门元")
        print("3. 确认分类特征中包含 '规模等级' 字段")
        print("4. 查看规模等级字段的值是否合理（小型/中型/大型）")
        print("5. 如需帮助，请提供数据样本截图")

SyntaxError: expected 'except' or 'finally' block (ipython-input-2953652964.py, line 389)